# Computing ephemerides of Solar System Objects

The purpose of this nootbok is to practice at querying API (Application Public Interface) of Solar System services.

We will use the generation of ephemerides as example, with two different services:
- [**Miriade**](https://ssp.imcce.fr/webservices/miriade) of the [IMCCE](https://www.imcce.fr/), directly from the API
- [**Ephemeris Service**](https://www.minorplanetcenter.net/iau/MPEph/MPEph.html) of the [Minor Planet Center](https://minorplanetcenter.net/), through an [`astroquery`](https://astroquery.readthedocs.io/en/latest/) wrapper

Other similar services exist and can be queried programmatically:
- [**Horizons**](https://ssd.jpl.nasa.gov/horizons/) from [JPL SSD](https://ssd.jpl.nasa.gov/): from their [API](https://ssd-api.jpl.nasa.gov/doc/horizons.html) or with an [`astroquery`](https://astroquery.readthedocs.io/en/latest/) [wrapper](https://astroquery.readthedocs.io/en/latest/jplhorizons/jplhorizons.html)
- [**Miriade**](https://ssp.imcce.fr/webservices/miriade) from the [IMCCE](https://www.imcce.fr/), using the [`astroquery`](https://astroquery.readthedocs.io/en/latest/) [wrapper](https://astroquery.readthedocs.io/en/latest/imcce/imcce.html#miriade-ephemeris-service) this time


In [ ]:
import os
import io
import requests
import json
from astropy.coordinates import Angle

import rocks
import pandas as pd
import time as t

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.projections

from astropy.coordinates import SkyCoord

from astroquery.mpc import MPC

## Define the parameters of the query

We define here the targets we want to observe, when, for how many epochs, separated by a time step, *etc*

In [2]:
# When we observe
epoch = "2024-01-01T00:00:00"
nbd = 180
step = "5d"

# What we observe
targets = ["Pallas", 3, "vesta", "2003 EL61"]

# Where are we observing from
observer = (
    "309"  # Paranal, see https://www.minorplanetcenter.net/iau/lists/ObsCodesF.html
)

## 1. Direct API query on IMCCE/Miriade

In [ ]:
# Service URL
url = "https://ssp.imcce.fr/webservices/miriade/api/ephemcc.php?"


# Query parameters
# The request package that we will use will create the full URL for us from the dictionary below
params = {
    "-name": targets[0],
    "-ep": epoch,
    "-nbd": nbd,
    "-step": step,
    "-mime": "json",
    "-observer": observer,
    "-tscale": "UTC",
    "-from": "Student_at_Les_Houches",
}

# Update the target of the query with its latest designation
# It is a safety step, to avoid issues
ids = rocks.id(targets[0])
params["-name"] = ids[0]

# Query the service
try:
    r = requests.post(url, params=params, timeout=2000)
except requests.exceptions.ReadTimeout:
    print("Request timed out")

# Convert the json results into a python dictionary
result = json.loads(r.content)

# Get ephemerides in a pandas DataFrame
eph = pd.DataFrame.from_dict(result["data"])

# Convert RA/Dec from hours/degrees to degrees
coords = SkyCoord(eph["RA"], eph["DEC"], unit=("hour", "deg"))
eph["RA_deg"] = coords.ra.degree
eph["DEC_deg"] = coords.dec.degree

# Plot the trajectory
fig, ax = plt.subplots()
ax.scatter(eph["RA_deg"], eph["DEC_deg"], label=targets[0])
ax.set_xlabel("Right Ascension (deg)")
ax.set_ylabel("Declination (deg)")
ax.grid()
ax.legend()

The result of the query contains several items, including information on the SSO, the signification of each columns, the unit, the reference frame, *etc*, see the different keys of the dictionary:

In [ ]:
result.keys()

In [ ]:
result["sso"]

In [ ]:
result["unit"]

etc.

We can easily script this for several targets

In [ ]:
fig, ax = plt.subplots()

for i, sso in enumerate(targets):
    # Update the target of the query, but check first its designation
    ids = rocks.id(sso)
    params["-name"] = ids[0]

    # Query the service
    t0 = t.time()
    try:
        r = requests.post(url, params=params, timeout=2000)
    except requests.exceptions.ReadTimeout:
        print("Request timed out")
    t1 = t.time()
    print(f"Request of ephemerides for {ids[0]} took {t1-t0:.1f} seconds")

    # Convert the json results into a python dictionary
    result = json.loads(r.content)

    # Get ephemerides in a pandas DataFrame
    eph = pd.DataFrame.from_dict(result["data"])

    # Convert RA/Dec from hours/degrees to degrees
    coords = SkyCoord(eph["RA"], eph["DEC"], unit=("hour", "deg"))
    eph["RA_deg"] = coords.ra.degree
    eph["DEC_deg"] = coords.dec.degree

    # Plot the trajectory
    ax.scatter(eph["RA_deg"], eph["DEC_deg"], s=4, label=params["-name"])

# Set up axes
ax.set_xlabel("Right Ascension (deg)")
ax.set_ylabel("Declination (deg)")
ax.grid()
ax.legend()

Whenever you want to use an API, have a quick look at the documentation to see what it can do and how to use it!
In the present case of Miriade, the documentation is available at https://ssp.imcce.fr/webservices/miriade/api/ephemcc/

For instance, you can easily switch to **ecliptic** coordinates (*rplane* keyword), change the list of reported columns ([*tcoor* keyword](https://ssp.imcce.fr/webservices/miriade/api/ephemcc/#output-results)), compute magnitude in different filters ([*colors* keyword](https://ssp.imcce.fr/webservices/miriade/howto/ephemcc/#howto-filters)), even in the thermal infrared
([*thermalFlux* keyword](https://ssp.imcce.fr/webservices/miriade/howto/ephemcc/#howto-thermalflux)).

## 2. Using astroquery wrapper to query MPC

In the example above, we used the `requests` module to directly interact with the API. We are in full control of what we query.

In some cases, someone already developped a python interface to the service, as illustrated below, thanks to [`astroquery`](https://astroquery.readthedocs.io/en/latest/). 

Depending on his/her needs, available time, coding capacities, *etc*, one may choose either of these two approaches.

In [ ]:
# Update the target of the query, but check first its designation
ids = rocks.id(targets[0])

t0 = t.time()
try:
    eph = MPC.get_ephemeris(
        ids[0], step=step, start=epoch, number=nbd, location=observer
    )
except requests.exceptions.ReadTimeout:
    print("Request timed out")
t1 = t.time()
print(f"Request of ephemerides for {ids[0]} took {t1-t0:.1f} seconds")

The result here is an [`astropy`](https://www.astropy.org/) [Table](https://docs.astropy.org/en/stable/table/) object. It is in essence a matrix, that can be accessed by rows/columns. Each column is a [`Quantity`](https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity): it has a unit.

In [ ]:
# Show first four rows
eph[:4]

An advantage of using a community-supported wrapper is that you do not have to figure out how to build the query nor how to decode the answer.

In [ ]:
# Plot the trajectory
fig, ax = plt.subplots()
ax.scatter(eph["RA"], eph["Dec"], label=ids[0])
ax.set_xlabel("Right Ascension (deg)")
ax.set_ylabel("Declination (deg)")
ax.grid()
ax.legend()

## Exercices

### Ex1. Plan your night of observations 

Let's start with a simple exercice, simply changing some parameters. Get the ephemerides for
- KBO Haumea
- On the night of 2024, February the 10th
- From La Silla observatory
- With local coordinates (azimuth and elevation)
- With a time step between points of 5 minutes
- (optional) Plot the elevation of the target as function of time

In [11]:
# Code here

### Ex2. Cartesian coodinates

You are interested in the geometry of encounter between your favorite space mission (e.g. [Lucy](https://science.nasa.gov/mission/lucy/)) and a flyby target (e.g., [Patroclus](https://ssp.imcce.fr/forms/ssocard/Patroclus)). In such case, you would need the position of the target in heliocentric cartesian coordinates.

- Compute the trajectory of asteroids Patroclus from 2020 to 2030 in heliocentric ecliptic cartesian coordinates using Miriade.
- Plot its trajectory in a ecliptic (x,y) plane.
- (optional) Overplot the motion of planets Earth and Jupiter over the same time period
- (optional) Overplot the motion of the comet 67P/Churyumov–Gerasimenko

In [12]:
# Code here!

<br><br><br><br><br><br><br><br>
Solutions comes here below
<br><br><br><br><br>
Don't look at the solutions without trying yourself!
<br><br><br><br>

<br><br><br><br>
Ready?

<br><br><br><br>
Not cheating?


### Solution Ex1. Plan your night of observations 

Let's start with a simple exercice, simply changing some parameters. Get the ephemerides for
- KBO Haumea
- On the night of 2024, February the 10th
- From La Silla observatory
- With local coordinates (azimuth and elevation)
- With a time step between points of 5 minutes
- (optional) Plot the elevation of the target as function of time

The same with the planets!

In [13]:
# Service URL
url = "https://ssp.imcce.fr/webservices/miriade/api/ephemcc.php?"


# Query parameters
params = {
    "-name": "Haumea",
    "-ep": "2024-02-10",
    "-nbd": (60 / 5) * 24,
    "-step": "5min",
    "-mime": "json",
    "-observer": "809",
    "-tcoor": 5,
    "-tscale": "UTC",
    "-output": "--jd",
    "-from": "Student_at_Les_Houches",
}

# Query the service
try:
    r = requests.post(url, params=params, timeout=2000)
except requests.exceptions.ReadTimeout:
    print("Request timed out")

# Convert the json results into a python dictionary
result = json.loads(r.content)

# Get ephemerides in a pandas DataFrame
eph = pd.DataFrame.from_dict(result["data"])

In [ ]:
# Plot elevation curves
fig, ax = plt.subplots(1, 2, figsize=(15, 5), gridspec_kw={"width_ratios": [2, 1]})

# Object is above the horizon
cond = eph["H"] > 0

# Plot the elevation as function of time
ax[0].scatter(
    eph.loc[cond, "Date"],
    eph.loc[cond, "H"],
    label=params["-name"] + "(above horizon)",
    s=4,
)
ax[0].scatter(
    eph.loc[~cond, "Date"],
    eph.loc[~cond, "H"],
    label=params["-name"] + "(below)",
    color="grey",
    s=4,
)

# Plot the path on the sky
ax[1].remove()
ax[1] = fig.add_subplot(1, 2, 2, projection="polar")
ax[1].scatter(
    np.radians(eph.loc[cond, "Az"]), eph.loc[cond, "H"], label=params["-name"], s=4
)
ax[1].set_ylim(90, 0)
ax[1].set_theta_zero_location("S")

# Set up axes
ax[0].set_xlabel("Time (JD)")
ax[0].set_ylabel("Elevation (deg)")
ax[0].grid()
ax[0].legend()
ax[1].set_xlabel("Azimuth (deg)")

### Ex2. Cartesian coodinates

You are interested in the geometry of encounter between your favorite space mission (e.g. [Lucy](https://science.nasa.gov/mission/lucy/)) and a flyby target (e.g., [Patroclus](https://ssp.imcce.fr/forms/ssocard/Patroclus)). In such case, you would need the position of the target in heliocentric cartesian coordinates.

- Compute the trajectory of asteroids Patroclus from 2020 to 2030 in heliocentric ecliptic cartesian coordinates using Miriade.
- Plot its trajectory in a ecliptic (x,y) plane.
- (optional) Overplot the motion of planets Earth and Jupiter over the same time period
- (optional) Overplot the motion of the comet 67P/Churyumov–Gerasimenko

Whenever you want to use an API, have a quick look at the documentation to see what it can do and how to use it!
In the present case of Miriade, the documentation is available at https://ssp.imcce.fr/webservices/miriade/api/ephemcc/

For instance, you can easily switch to **ecliptic** coordinates (*rplane* keyword), change the list of reported columns ([*tcoor* keyword](https://ssp.imcce.fr/webservices/miriade/api/ephemcc/#output-results)), compute magnitude in different filters ([*colors* keyword](https://ssp.imcce.fr/webservices/miriade/howto/ephemcc/#howto-filters)), even in the thermal infrared
([*thermalFlux* keyword](https://ssp.imcce.fr/webservices/miriade/howto/ephemcc/#howto-thermalflux)).

In [15]:
# Service URL
url = "https://ssp.imcce.fr/webservices/miriade/api/ephemcc.php?"


# Query parameters
params = {
    "-name": "Patroclus",
    "-ep": "2020-01-01",
    "-nbd": 360,
    "-step": "10d",
    "-mime": "json",
    "-observer": "@sun",
    "-rplane": 2,
    "-tcoor": 2,
    "-output": "--jd",
    "-from": "Student_at_Les_Houches",
}

# Query the service
try:
    r = requests.post(url, params=params, timeout=2000)
except requests.exceptions.ReadTimeout:
    print("Request timed out")

# Convert the json results into a python dictionary
result = json.loads(r.content)

# Get ephemerides in a pandas DataFrame
patroclus = pd.DataFrame.from_dict(result["data"])

In [ ]:
# Plot elevation curves
fig, ax = plt.subplots()

# The Sun!
ax.scatter(0, 0, marker="*", s=100, color="orange", label="Sun")

# --------------------------------------------------------------------------------
# Plot the trajectory
im = ax.scatter(
    patroclus["px"], patroclus["py"], c=patroclus["Date"], label="Patroclus", s=4
)

cbar = fig.colorbar(im)

# --------------------------------------------------------------------------------
# Set up axes
ax.set_xlabel("Ecliptic X (au)")
ax.set_ylabel("Ecliptic Y (au)")
ax.set_aspect("equal")
ax.set_xlim(-7, 7)
ax.legend(loc="upper right")

The same with the planets!

In [ ]:
# Plot elevation curves
fig, ax = plt.subplots()


# --------------------------------------------------------------------------------
# The Sun!
ax.scatter(0, 0, marker="*", s=100, color="orange", label="Sun")

# Plot the trajectory
im = ax.scatter(
    patroclus["px"], patroclus["py"], c=patroclus["Date"], label="Patroclus", s=20
)
cbar = fig.colorbar(im)


# --------------------------------------------------------------------------------
# Add the comet

# Query the service - Simply changing the target
params["-name"] = "c:67P"

try:
    r = requests.post(url, params=params, timeout=2000)
except requests.exceptions.ReadTimeout:
    print("Request timed out")

# Convert the json results into a python dictionary
result = json.loads(r.content)

# Get ephemerides in a pandas DataFrame
comet = pd.DataFrame.from_dict(result["data"])

# Plot the trajectory
ax.scatter(comet["px"], comet["py"], c=comet["Date"], label="67P", marker="d", s=20)


# --------------------------------------------------------------------------------
# Add planets
for i, p in enumerate(["Mercury", "Venus", "Earth", "Mars", "Jupiter"]):
    # Query parameters
    params = {
        "-name": f"p:{p}",
        "-ep": "2020-01-01",
        "-nbd": 360,
        "-step": "10d",
        "-mime": "json",
        "-observer": "@sun",
        "-rplane": 2,
        "-tcoor": 2,
        "-output": "--jd",
        "-from": "Student_at_Les_Houches",
    }

    # Query the service *if* the ephemerides is not already downloaded
    if not os.path.exists(f"{p}.csv"):
        try:
            r = requests.post(url, params=params, timeout=2000)
        except requests.exceptions.ReadTimeout:
            print("Request timed out")

        # Convert the json results into a python dictionary
        result = json.loads(r.content)

        # Get ephemerides in a pandas DataFrame
        planet = pd.DataFrame.from_dict(result["data"])
        planet.to_csv(f"{p}.csv", index=False)

    else:
        planet = pd.read_csv(f"{p}.csv")

    # Overplot the planet
    ax.plot(planet["px"], planet["py"], label=p)

# --------------------------------------------------------------------------------
# Set up axes
ax.set_xlabel("Ecliptic X (au)")
ax.set_ylabel("Ecliptic Y (au)")
ax.set_aspect("equal")
ax.set_xlim(-7, 7)
ax.legend(loc="lower right")